In [2]:
import numpy as np
from scipy  import stats
import math


\begin{align*}
L_i &= \{C_i, C_{i+1}\} \quad for \quad i = 1,\ldots, T-1\\
\psi_{L_i}(C_i, C_{i+1}) &= P(C_{i+1} \mid C_i)\\
M_{i}  &= \{C_i, Z_{i,1}, Z_{i,2}, \ldots, Z_{i,n}\} \quad for \quad i =1,\ldots, T\\
\psi_{M_i}(C_i, Z_{i,1}, Z_{i,2}, \ldots, Z_{i,n})&= \prod_{j = 1}^n P(Z_{i,j} \mid C_i)\\
N_{i,j} &= \{Z_{i,j}, X_{i,j}\} \quad for \quad i = 1, \ldots, T, j = 1,\ldots, n\\
\psi_{N_{i,j}}(Z_{i,j}, X_{i,j}) &= P(X_{i,j} \mid Z_{i,j})
\end{align*}

\begin{align*}
 M_1  \to L_1 \to \ldots \to M_{i+1} \to L_{i+1} \ldots \to M_{T}
\end{align*}

In [222]:
class Variables:
    def __init__(self, T = 10, n = 2, alpha = 0.9, beta = 0.2, gamma = 0.1, lamb0 = 1, lamb1  = 5) -> None:
        #Customizable values
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.lamb0 = lamb0
        self.lamb1 = lamb1
        self.T = T
        self.n = n

        #--- Given probabilities---
        self.prob_C1 = np.array([0,0,1])
        self.Z_given_C = np.array([[self.alpha, 1-self.alpha],[1-self.alpha, self.alpha], [0.5, 0.5]])  #(C, Z)
        self.rates = [self.lamb0, self.lamb1]
        self.Gamma = np.array([[1-self.gamma, 0, self.gamma], #(Ci, Ci_plus1)
                          [0, 1-self.gamma, self.gamma], 
                          [self.beta/2, self.beta/2, 1-self.beta]])
        self.alpha_C = np.array([1-self.alpha, self.alpha, 0.5])
        

    def sim_data(self):
        ck = np.arange(3)   
    
        C_transition = [
            stats.rv_discrete(values=(ck,self.Gamma[0,])),#P(C_t |C_{t-1} = 0)
            stats.rv_discrete(values=(ck,self.Gamma[1,])),#P(C_t |C_{t-1} = 1)
            stats.rv_discrete(values=(ck,self.Gamma[2,])),#P(C_t |C_{t-1} = 2)
        ]
        
        C = np.zeros(self.T, np.int64)
        C[0] = 2
        for i in range(self.T-1):
            C[i+1] = C_transition[C[i]].rvs()
        self.C = C
        
        self.Z = stats.bernoulli([self.Z_given_C[c,1] for c in C]).rvs(size=[self.n,self.T])
        self.X = stats.poisson(np.where(self.Z, self.lamb1, self.lamb0)).rvs() #(n, T)
        return 
    
    def find_P_X_given_C(self):
        self.P_X_given_Z = np.array([stats.poisson.pmf(k = self.X, mu = self.lamb0), #All of the potentials
                            stats.poisson.pmf(k = self.X, mu = self.lamb1)]) #(Z, n, T)
        
        self.P_X_given_C = np.einsum("ZnT, CZ -> TnC",self.P_X_given_Z, self.Z_given_C) #(Time, n, C)

    def upwards_passing(self):
        #Defining properties
        delta_Mi_to_Li = np.empty(shape = (self.T-1, 3)) #(Time, Ci)
        delta_Li_to_Mi_plus = np.empty(shape = (self.T-1, 3)) #(C_{i+1})
        belief_Li = np.empty(shape = (self.T, 3, 3)) #(T, C_i, C_{i+1})
    
        for i in range(self.T-1):
            #----------------Sending messages-------------
            #finding message Ci, Zi -> Ci+, Ci+
            if i == 0:
                delta_Mi_to_Li[i] = np.prod(self.P_X_given_C[i,:,:], axis = 0)*self.prob_C1 #C1
            else:
                delta_Mi_to_Li[i] = np.prod(self.P_X_given_C[i,:,:], axis = 0)*delta_Li_to_Mi_plus[i-1] #Ci

            #normalizing
            delta_Mi_to_Li[i] /= np.sum(delta_Mi_to_Li[i])

            #Finding belief Li
            belief_Li[i] = np.einsum("cC,c -> cC", self.Gamma, delta_Mi_to_Li[i]) #c old (C_{i}), C new (C_{i+1})
            #normalizing
            belief_Li[i] /= np.sum(belief_Li[i])
            
            #finding message Ci Ci+ -> Ci+, Zi+
            delta_Li_to_Mi_plus[i] = np.sum(belief_Li[i], axis=0) #Summing out c old
            
            #normalizing
            delta_Li_to_Mi_plus[i] /= np.sum(delta_Li_to_Mi_plus[i])

        self.delta_Mi_to_Li = delta_Mi_to_Li
        self.delta_Li_to_Mi_plus = delta_Li_to_Mi_plus
        self.belief_Li = belief_Li
        return 
    
    def backwards_passing(self):
        delta_Mi_plus_to_Li = np.empty(shape= (self.T-1, 3)) #T, C
        delta_Li_to_Mi = np.empty(shape= (self.T -1, 3)) #T, C
        updated_belief_Li = np.empty(shape = (self.T-1, 3, 3)) #T, c old (C_{i}), C new (C_{i+1})       

        for i in range(self.T-1):
            #---- SENDING MESSAGES ----- #
            #find downwards message Mi+ -> Li
            j = self.T -1 - i -1 #finding opposite direvtion
            if i == 0: #then j = T-2, aka the last entry index in messages
                delta_Mi_plus_to_Li[j] = np.prod(self.P_X_given_C[j+1,:,:], axis = 0) #(C)
            else:
                delta_Mi_plus_to_Li[j] = np.prod(self.P_X_given_C[j+1,:,:], axis = 0)*delta_Li_to_Mi[j+1] #(C)
            #normalize
            delta_Mi_plus_to_Li[j] /= np.max(delta_Mi_plus_to_Li[j])

            # find message Li -> Mi
            delta_Li_to_Mi[j] = np.einsum("cC,C -> c", self.Gamma, delta_Mi_plus_to_Li[j]) #Summing out Ci+

            #normalize
            delta_Li_to_Mi[j] /= np.sum(delta_Li_to_Mi[j])

            #----Update clique belief of Li

            #find belief
            updated_belief_Li[j] = np.einsum("cC, C -> cC", 
                                             np.einsum("cC, c -> cC", self.Gamma, self.delta_Mi_to_Li[j]), 
                                             delta_Mi_plus_to_Li[j]) #old c (ci), new C (C_{i+1})
            #normalize
            updated_belief_Li[j] /= np.sum(updated_belief_Li[j])
            


        self.updated_belief_Li = updated_belief_Li
        self.delta_Li_to_Mi = delta_Li_to_Mi
        self.delta_Mi_plus_to_Li = delta_Mi_plus_to_Li
        
    def find_C_probabilites(self):
        #Define structures
        C_prob = np.empty(shape = (self.T, 3, 2)) #Time, C, the two methods

        for i in range(self.T-1):
            #
            guess_0 = np.sum(self.updated_belief_Li[i], axis=1) #summing out C_{i+1}
            guess_1 = np.sum(self.updated_belief_Li[i], axis= 0) #summing out C_i
            C_prob[i,:,0] = guess_0     #Time, C_val, left_cliques
            C_prob[i+1,:,1] = guess_1   #Time, C_val, right_cliques
        
        #Filling in gaps
        C_prob[0,:,1] = C_prob[0,:,0]   #First probability
        C_prob[-1,:,0] = C_prob[-1,:,1] #last probability


        self.C_probs = C_prob
        self.C_prob = C_prob[:,:,0]
        return 

    def find_Z_probabilities(self):
        delta_Mi_to_Nij = np.empty(shape = (self.T, 2)) #Time, Z_val: This message is the same accross neurons, it is only time dependent

        #Finding first T-1 messages
        for i in range(self.T-1):
            delta_Mi_to_Nij[i] = np.einsum("c, cZ -> Z", self.delta_Li_to_Mi[i], self.Z_given_C)

        #Finding last message
        delta_Mi_to_Nij[-1] = np.einsum("C, CZ -> Z", self.delta_Li_to_Mi_plus[-1], self.Z_given_C)      
        
        belief_Zij = np.einsum("TZ, ZnT -> nTZ", delta_Mi_to_Nij, self.P_X_given_Z) #(T,Z) x (Z, n, T) -> (n, T,Z)
        
        row_sums = np.sum(belief_Zij, axis=-1)

        #normalize
        belief_Zij/= row_sums[..., np.newaxis]

        self.belief_Zij = belief_Zij
        self.delta_MI_to_Nij = delta_Mi_to_Nij
        return 

    def find_probabilities(self):
        self.sim_data()
        self.find_P_X_given_C() #time, C
        self.upwards_passing()
        self.backwards_passing()
        self.find_C_probabilites()
        self.find_Z_probabilities()
        

    def variable_prediction_precision(self):
        self.find_probabilities()

        self.accuracy_Z = [(self.Z == z) - self.belief_Zij[:,:,z] for z in range(2)] #Average distance from true value for Z
        self.accuracy_C = [(self.C == i) - self.C_prob[:,i] for i in range(3)] #Average distance from true value for C's
        return self.accuracy_C, self.accuracy_Z

    def message_pass(self):
        self.find_P_X_given_C() #time, C
        self.upwards_passing()
        self.backwards_passing()
        self.find_C_probabilites()
        self.fin
        return self.C_prob
        

def check_simulation(N):
    val_c = np.empty(shape = (N, 3, variables.T)) #N, C, T
    val_Z = np.empty(shape = (N, 2, variables.n, variables.T)) #N, Z, n, T
    for i in range(N):
        sim_i = Variables(T = variables.T, n = variables.n)
        val_c[i], val_Z[i] = sim_i.variable_prediction_precision()

    print("Did N = ", N, " simulations with n = ", variables.n, " and T = ", variables.T)
    print("greatest error for C is:", np.around(np.max(np.abs(np.mean(val_c, axis = 0)))*100, decimals = 2), "%")
    print("greatest error for Z is:", np.around(np.max(np.abs(np.mean(val_Z, axis = 0)))*100, decimals = 2), "%")
    print("Accuracy for estimates of C: \n", np.around(np.mean(val_c, axis = 0), decimals= 2))
    print("Accuracy for estimates of Z: \n", np.around(np.mean(val_Z, axis = 0), decimals= 2))
    return 

variables = Variables(T = 100, n = 100)
check_simulation(10000)

Did N =  10000  simulations with n =  100  and T =  100
greatest error for C is: 0.03 %
greatest error for Z is: 1.06 %
Accuracy for estimates of C: 
 [[ 0.  0.  0.  0.  0. -0. -0.  0. -0. -0.  0. -0.  0. -0. -0. -0.  0. -0.
   0.  0. -0. -0.  0.  0.  0.  0. -0. -0.  0.  0. -0.  0. -0. -0.  0. -0.
   0. -0.  0. -0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0. -0.  0. -0.  0.
  -0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0. -0.  0.  0. -0. -0. -0.  0.
  -0.  0.  0. -0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0. -0. -0.  0. -0.]
 [ 0. -0.  0. -0.  0.  0. -0.  0. -0. -0.  0. -0.  0.  0.  0.  0.  0.  0.
   0.  0. -0. -0.  0. -0.  0.  0. -0. -0.  0. -0.  0.  0.  0. -0.  0. -0.
   0.  0. -0. -0. -0. -0. -0.  0. -0. -0.  0.  0.  0.  0.  0. -0.  0. -0.
   0. -0.  0.  0. -0. -0.  0.  0. -0. -0. -0.  0. -0.  0. -0.  0.  0. -0.
   0.  0. -0.  0.  0.  0.  0.  0.  0. -0. -0.  0.  0.  0. -0.  0. -0.  0.
   0. -0.  0. -0.  0.  0.  0.  0. -0.  0.]
 [ 0.  0. -0.  0. -0. -

In [113]:
#generate data
variables.find_P_X_given_C() #time, C
variables.upwards_passing()
variables.backwards_passing()
variables.find_C_probabilites()
variables.find_Z_probabilities()


In [131]:
variables.variable_prediction_precision()

([array([ 0.        , -0.06267557, -0.23775447, -0.6874107 ,  0.18786753,
          0.18284927,  0.06490248,  0.03590944,  0.03895488,  0.07786069]),
  array([ 0.00000000e+00, -1.72162887e-02, -4.84316234e-03, -4.61663851e-04,
         -2.32322622e-04, -4.87789731e-04, -3.33884254e-05, -2.42311857e-05,
         -6.35683385e-05, -9.21548932e-04]),
  array([ 0.        ,  0.07989186,  0.24259763,  0.68787236, -0.18763521,
         -0.18236148, -0.0648691 , -0.03588521, -0.03889131, -0.07693914])],
 [array([[-0.02393668, -0.04571298,  0.00369391, -0.97889962,  0.00616067,
           0.01719766,  0.00372416,  0.00442119,  0.03703998,  0.01800637],
         [ 0.06124708, -0.85689435, -0.00344048,  0.00429254,  0.03006253,
          -0.08377579,  0.00372416,  0.02172181,  0.00763421,  0.08398294]]),
  array([[ 0.02393668,  0.04571298, -0.00369391,  0.97889962, -0.00616067,
          -0.01719766, -0.00372416, -0.00442119, -0.03703998, -0.01800637],
         [-0.06124708,  0.85689435,  0.003440

Did N =  1000  simulations with n =  2  and T =  10
Accuracy for estimates of C: 
 [[ 0.    0.01 -0.01 -0.   -0.01  0.    0.01  0.01  0.01 -0.  ]
 [ 0.    0.   -0.   -0.   -0.   -0.01 -0.01  0.    0.    0.01]
 [ 0.   -0.01  0.01  0.    0.01  0.    0.   -0.01 -0.01 -0.01]]
Accuracy for estimates of Z: 
 [[[-0.    0.02  0.    0.01  0.    0.   -0.01  0.01  0.   -0.01]
  [ 0.01  0.    0.02  0.    0.    0.    0.    0.02 -0.   -0.01]]

 [[ 0.   -0.02 -0.   -0.01 -0.   -0.    0.01 -0.01 -0.    0.01]
  [-0.01 -0.   -0.02 -0.   -0.   -0.   -0.   -0.02  0.    0.01]]]


In [97]:
print(variables.Z)
np.around(variables.belief_Zij, decimals =2)

[[1 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]]


array([[[0.41, 0.59],
        [0.99, 0.01]],

       [[0.04, 0.96],
        [0.99, 0.01]],

       [[0.98, 0.02],
        [0.  , 1.  ]],

       [[0.98, 0.02],
        [0.69, 0.31]],

       [[1.  , 0.  ],
        [0.98, 0.02]],

       [[0.98, 0.02],
        [0.92, 0.08]],

       [[0.91, 0.09],
        [1.  , 0.  ]],

       [[1.  , 0.  ],
        [0.98, 0.02]],

       [[0.99, 0.01],
        [0.97, 0.03]],

       [[0.92, 0.08],
        [1.  , 0.  ]]])

In [137]:
def upwards_passing():
    delta_Mi_to_Li = np.empty(shape = (variables.T-1, 3)) #(Time, Ci)
    delta_Li_to_Mi_plus = np.empty(shape = (variables.T-1, 3)) #(C_{i+1})
    belief_Li = np.empty(shape = (variables.T, 3, 3)) #(T, C_i, C_{i+1})
    
    
    for i in range(variables.T-1):
        #finding message Ci, Zi -> Ci+, Ci+
        if i == 0:
            delta_Mi_to_Li[i] = variables.P_X_given_C[i]*variables.prob_C1 #C1
        else:
            delta_Mi_to_Li[i] = variables.P_X_given_C[i]*delta_Li_to_Mi_plus[i-1] #Ci
        

        #normalizing
        delta_Mi_to_Li[i] /= np.sum(delta_Mi_to_Li[i])

        #Finding belief Li
        belief_Li[i] = np.einsum("cC,c -> cC", variables.Gamma, delta_Mi_to_Li[i]) #c old (C_{i}), C new (C_{i+1})
        belief_Li[i] /= np.sum(belief_Li[i])
        #finding message Ci Ci+ -> Ci+, Zi+
        delta_Li_to_Mi_plus[i] = np.sum(belief_Li[i], axis=0)
        #normalizing
        delta_Li_to_Mi_plus[i] /= np.sum(delta_Li_to_Mi_plus[i])

    return delta_Mi_to_Li, delta_Li_to_Mi_plus, belief_Li
        


In [169]:
def backwards_passing():
    delta_Mi_plus_to_Li = np.empty(shape= (variables.T-1, 3)) #T, C
    delta_Li_to_Mi = np.empty(shape= (variables.T -1, 3)) #T, C
    updated_belief_Li = np.empty(shape = (variables.T-1, 3, 3)) #T, c old (C_{i}), C new (C_{i+1})

    for i in range(variables.T-1):
        #find downwards message Mi+ -> Li
        j = variables.T -1 - i -1 #finding opposite direvtion
        if i == 0:
            delta_Mi_plus_to_Li[i] = variables.delta_Li_to_Mi_plus[j]*variables.P_X_given_C[j] #(C)
        else:
            delta_Mi_plus_to_Li[i] = variables.delta_Li_to_Mi_plus[j]*delta_Li_to_Mi[i]*variables.P_X_given_C[j] #(C)
        
        #normalize
        delta_Mi_plus_to_Li[i] /= np.max(delta_Mi_plus_to_Li[i])

        #find belief
        updated_belief_Li[i] = np.einsum("cC, C -> cC", variables.belief_Li[j], delta_Mi_plus_to_Li[i]/variables.delta_Li_to_Mi_plus[j])
        # find message Li -> Mi

        delta_Li_to_Mi[i] = np.sum(updated_belief_Li[i], axis=1)
        
        #normalize
        delta_Li_to_Mi[i] /= np.sum(delta_Li_to_Mi[i])

    return updated_belief_Li, delta_Li_to_Mi, delta_Mi_plus_to_Li
        
backwards_passing()

(array([[[9.67278499e-01, 0.00000000e+00, 8.70204697e-02],
         [0.00000000e+00, 2.12573210e-02, 3.08772003e-03],
         [3.27215013e-02, 2.02662653e-02, 2.11951066e-01]],
 
        [[9.66758614e-01, 0.00000000e+00, 2.92033582e-03],
         [0.00000000e+00, 1.15564430e-04, 7.26437152e-05],
         [3.32413860e-02, 1.59742326e-04, 7.22980135e-03]],
 
        [[9.54996163e-01, 0.00000000e+00, 1.45842588e-02],
         [0.00000000e+00, 5.16637059e-05, 5.10100333e-04],
         [4.50038373e-02, 6.96084929e-05, 4.94839978e-02]],
 
        [[9.23703465e-01, 0.00000000e+00, 6.86015641e-02],
         [0.00000000e+00, 6.10409292e-03, 1.00718544e-02],
         [7.62965350e-02, 3.43413969e-03, 4.07979890e-01]],
 
        [[8.86078705e-01, 0.00000000e+00, 2.85045759e-02],
         [0.00000000e+00, 9.67640743e-03, 7.13747334e-03],
         [1.13921295e-01, 4.96840333e-03, 2.63863728e-01]],
 
        [[1.40334713e-01, 0.00000000e+00, 5.05755735e-02],
         [0.00000000e+00, 1.13191062e-01,